# Homework 2: Higher Order Functions

## Required questions

### Q1. Product

```python
def product(n, term):
    """Return the product of the first n terms in a sequence.
    n    -- a positive integer
    term -- a function that takes one argument

    >>> product(3, identity)  # 1 * 2 * 3
    6
    >>> product(5, identity)  # 1 * 2 * 3 * 4 * 5
    120
    >>> product(3, square)    # 1^2 * 2^2 * 3^2
    36
    >>> product(5, square)    # 1^2 * 2^2 * 3^2 * 4^2 * 5^2
    14400
    >>> product(3, increment) # (1+1) * (2+1) * (3+1)
    24
    >>> product(3, triple)    # 1*3 * 2*3 * 3*3
    162
    """
    if n==0:
        return 0
    else:
        i = 1
        prod = 1
        while i <= n:
            prod *= term(i)
            i += 1
        return prod
```
Although it was not necessary, I added a check for `n==0`. Notice that `prod = 1` at the begining.

### Q2. Accumulate

```python
def accumulate(combiner, base, n, term):
    """Return the result of combining the first n terms in a sequence and base.
    The terms to be combined are term(1), term(2), ..., term(n).  combiner is a
    two-argument commutative, associative function.

    >>> accumulate(add, 0, 5, identity)  # 0 + 1 + 2 + 3 + 4 + 5
    15
    >>> accumulate(add, 11, 5, identity) # 11 + 1 + 2 + 3 + 4 + 5
    26
    >>> accumulate(add, 11, 0, identity) # 11
    11
    >>> accumulate(add, 11, 3, square)   # 11 + 1^2 + 2^2 + 3^2
    25
    >>> accumulate(mul, 2, 3, square)    # 2 * 1^2 * 2^2 * 3^2
    72
    >>> accumulate(lambda x, y: x + y + 1, 2, 3, square)
    19
    >>> accumulate(lambda x, y: 2 * (x + y), 2, 3, square)
    58
    >>> accumulate(lambda x, y: (x + y) % 17, 19, 20, square)
    16
    """
    if n==0:
        return base
    else:
        i = 1
        current_result = base
        while i <= n:
            current_result = combiner(current_result, term(i))
            i += 1
        return current_result
```
Iteratively combine the `current_result` with `term(i)` to create a new `current_result`. Another option is to recursively call itself changing `-1` the current value of `n` in each call.
```python
def accumulate(combiner, base, n, term):
    """Return the result of combining the first n terms in a sequence and base.
    The terms to be combined are term(1), term(2), ..., term(n).  combiner is a
    two-argument commutative, associative function.

    >>> accumulate(add, 0, 5, identity)  # 0 + 1 + 2 + 3 + 4 + 5
    15
    >>> accumulate(add, 11, 5, identity) # 11 + 1 + 2 + 3 + 4 + 5
    26
    >>> accumulate(add, 11, 0, identity) # 11
    11
    >>> accumulate(add, 11, 3, square)   # 11 + 1^2 + 2^2 + 3^2
    25
    >>> accumulate(mul, 2, 3, square)    # 2 * 1^2 * 2^2 * 3^2
    72
    >>> accumulate(lambda x, y: x + y + 1, 2, 3, square)
    19
    >>> accumulate(lambda x, y: 2 * (x + y), 2, 3, square)
    58
    >>> accumulate(lambda x, y: (x + y) % 17, 19, 20, square)
    16
    """
    if n==0:
        return base
    return combiner( term(n), accumulate(combiner, base, n-1, term) )
```
Implementation of `summation` and `product`:
```python
def summation_using_accumulate(n, term):
    """Returns the sum of term(1) + ... + term(n). The implementation
    uses accumulate.

    >>> summation_using_accumulate(5, square)
    55
    >>> summation_using_accumulate(5, triple)
    45
    """
    return accumulate(add, 0, n, term)

def product_using_accumulate(n, term):
    """An implementation of product using accumulate.

    >>> product_using_accumulate(4, square)
    576
    >>> product_using_accumulate(6, triple)
    524880
    """
    return accumulate(mul, 1, n, term)
```

### Q3. Make Repeater

Idea: iteratively apply `f` from `i=0` up to `i=n`.
```python
def make_repeater(f, n):
    """Return the function that computes the nth application of f.

    >>> add_three = make_repeater(increment, 3)
    >>> add_three(5)
    8
    >>> make_repeater(triple, 5)(1) # 3 * 3 * 3 * 3 * 3 * 1
    243
    >>> make_repeater(square, 2)(5) # square(square(5))
    625
    >>> make_repeater(square, 4)(5) # square(square(square(square(5))))
    152587890625
    >>> make_repeater(square, 0)(5) # Yes, it makes sense to apply the function zero times!
    5
    """
    def repeater(x):
        current_f = x
        i = 1
        while i<=n:
            current_f = f(current_f)
            i += 1
        return current_f
    return repeater
```

### Q4. Church Numerals

Let us first understand how `zero` and `successor` work:
```python
def zero(f):
    return lambda x: x

def successor(n):
    return lambda f: lambda x: f(n(f)(x))
```
- Calling `zero(f)` with any function returns the identity (and does not use the function `f`). 
- Calling `one = successor(zero)` returns a function whose argument is another function `f`. Now, calling `one(func)` returns a function of a single variable. To see why, calling to `one(func)` is equivalent to `func(zero(func)(*))`, which is equal to `func(identity(*))` as `zero(func)(*)` is the identity, i.e., always returns `*`. Therefore we end up with `func(*)`.
- Calling `two = successor(one)` returns a function whose argument is another function `f`. Now, calling `two(func)` returns a function of a single variable. To see why, calling to `two(func)` is equivalent to `func(one(func)(*))` which equals to `func(func(*))`, i.e., it applies twice the function.

The implementation is direct:
```python
def one(f):
    """Church numeral 1: same as successor(zero)"""
    return lambda x: f(x)

def two(f):
    """Church numeral 2: same as successor(successor(zero))"""
    return lambda x: f(f(x))
```

To convert Church numerals into integers we must count how many times we call f. That is the same as creating a function which adds one to a value each time is called. Thus, starting from 0, if we call that function with `one`, it adds 1 to 0.

```python
def church_to_int(n):
    """Convert the Church numeral n to a Python integer.

    >>> church_to_int(zero)
    0
    >>> church_to_int(one)
    1
    >>> church_to_int(two)
    2
    >>> church_to_int(three)
    3
    """
    def add_one(x):
        return x+1
    return n(add_one)(0)
```
Nevertheless it is still needed the existence of the integers 0 and 1 to generate 0 and 1! Not very useful.

Let us implement the sum of two Church numerals to give a Church numeral. As we have seen a Church numeral is a HOF of the form `c_n(f)(x)`, with `f` a function which we would like to apply `n` times to `x`. We would like to apply `f` `n+m` times to `x`, which is the Church numeral n+m. We can apply `m` times `f` to the result of applying `n` times `f` to `x`, i.e, `m(f)( result of applying n times f to x ) = m(f)( n(f)(x) )`, which is a Church numeral if we make `f` and `x` arguments:
```python
def add_church(m, n):
    """Return the Church numeral for m + n, for Church numerals m and n.

    >>> church_to_int(add_church(two, three))
    5
    >>> church_to_int(add_church(one, two))
    3
    >>> church_to_int(add_church(zero, two))
    2
    """
    return lambda f: lambda x: m(f)(n(f)(x))
```

Let us implement the multiplication of two Church numerals. When we multiply `m*n` we want to apply `m` times `n`, therefore the function of `m` now must be `n`. The function of `f` is the function we want to apply `m*n` times.
```python
def mul_church(m, n):
    """Return the Church numeral for m * n, for Church numerals m and n.

    >>> four = successor(three)
    >>> church_to_int(mul_church(two, three))
    6
    >>> church_to_int(mul_church(three, four))
    12
    >>> church_to_int(mul_church(one, four))
    4
    >>> church_to_int(mul_church(zero, two))
    0
    """
    return lambda f: lambda x: m(n(f))(x)
```

I don't understand the `pow` solution.